# Определение токсичности комментариев

Необходимо обучить модель, которая будет классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок. В качестве метрики качества возьмем F1-меру. 

Данные находятся в файле `toxic_comments.csv` (датасет не прилагается). Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df = pd.read_csv('toxic_comments.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Выполним лемматизацию текстов: приведем буквы к нижнему регистру, удалим все символы, кроме букв латинского алфавита, непосредственно лемматизацию выполним с помощью WordNetLemmatizer. Результат сохраним в признаке lemm_text:

In [3]:
corpus = df['text'].str.lower().values
corpus_cleared = [re.sub(r'[^a-z]', ' ', text) for text in corpus]

lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('wordnet')
corpus_lemmatized = [' '.join([lemmatizer.lemmatize(word) for word in (nltk.word_tokenize(text))]) for text in corpus_cleared]

df['lemm_text'] = corpus_lemmatized
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aleksandr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Aleksandr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


Создадим обучающую и тестовую выборки. Классы разбалансированы в пропорции 9:1

In [4]:
train_corpus, test_corpus, train_target, test_target = train_test_split(df['lemm_text'].values, df['toxic'], test_size=.25)
print('Среднее целевого признака в обучающей выборке:', round(train_target.mean(), 4))
print('Среднее целевого признака в тестовой выборке:', round(test_target.mean(), 4))

Среднее целевого признака в обучающей выборке: 0.1013
Среднее целевого признака в тестовой выборке: 0.1027


Выполним векторизацию текстов с помощью процедуры TF-IDF. Слова, не несущие информации о тональности текста (stopwords) в векторизации не учитываем:

In [5]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
vectorizer = TfidfVectorizer(stop_words=stopwords) 

train_tf_idf = vectorizer.fit_transform(train_corpus) #fit_transform only on train sample!
test_tf_idf = vectorizer.transform(test_corpus)

print('Размер матрицы TF-IDF на обучающей выборке:', train_tf_idf.shape)
print('Размер матрицы TF-IDF на тестовой выборке:', test_tf_idf.shape)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aleksandr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Размер матрицы TF-IDF на обучающей выборке: (119678, 133670)
Размер матрицы TF-IDF на тестовой выборке: (39893, 133670)


Задачу можно было попробовать решить с помощью метода мешок слов (bag of words):

    count_vect = CountVectorizer(stop_words=stopwords)
    train_bow = count_vect.fit_transform(train_corpus)
    test_bow = count_vect.transform(test_corpus)

Но использование этого метода дало заведомо меньшие значения метрики F1. Использование биграмм значительно увеличило размеры матриц (а вместе с ним и время обучения), но лучших показателей не дало: по всей видимости тональность текста определяется отельными словами. По этой причине в работе использованы только униграммы.

## Обучение

Для учета дисбаланса классов и получения приемлемых результатов предсказаний моделей будем использовать функцию, определяющую наилучшее значение порога классификации на основе метрики F1, вычисленной на обучающей выборке:

In [6]:
def best_threshold(model, train_features, train_target):
    best_f1_score = 0
    best_threshold = 0
    for threshold in np.arange(0, 1, .05):
        cv_probabilities = cross_val_predict(model, train_tf_idf, train_target, method='predict_proba', cv=2)
        predictions = (cv_probabilities >= threshold)[:,1].astype(int)
        if f1_score(train_target, predictions) > best_f1_score:
            best_f1_score = f1_score(train_target, predictions)
            best_threshold = threshold

    print('Наилучшее значение порога:', best_threshold.round(2))
    print('Наилучшее значение метрики F1 на обучающей выборке:', best_f1_score.round(4))
    return best_threshold

In [7]:
model = LogisticRegression(max_iter=1000)
threshold = best_threshold(model, train_tf_idf, train_target)
model.fit(train_tf_idf, train_target)
predictions = (model.predict_proba(test_tf_idf) >= threshold)[:,1].astype(int)
print('Метрика F1 на тестовой выборке:', round(f1_score(test_target, predictions), 4))

Наилучшее значение порога: 0.2
Наилучшее значение метрики F1 на обучающей выборке: 0.7577
Метрика F1 на тестовой выборке: 0.7803


In [8]:
model = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
threshold = best_threshold(model, train_tf_idf, train_target)
model.fit(train_tf_idf, train_target)
predictions = (model.predict_proba(test_tf_idf) >= threshold)[:,1].astype(int)
print('Метрика F1 на тестовой выборке:', round(f1_score(test_target, predictions), 4))

Наилучшее значение порога: 0.25
Наилучшее значение метрики F1 на обучающей выборке: 0.7587
Метрика F1 на тестовой выборке: 0.7653


In [9]:
model = CatBoostClassifier(verbose=40, iterations=200, eval_metric='F1')
model.fit(train_tf_idf, train_target)
predictions = model.predict(test_tf_idf) #CatBoost функцию с кросс-валидацией не осилил - слишком долго
print('Метрика F1 на тестовой выборке:', round(f1_score(test_target, predictions), 4))

Learning rate set to 0.347698
0:	learn: 0.4688563	total: 1.27s	remaining: 4m 13s
40:	learn: 0.6871345	total: 40.8s	remaining: 2m 38s
80:	learn: 0.7455137	total: 1m 18s	remaining: 1m 56s
120:	learn: 0.7672883	total: 1m 57s	remaining: 1m 16s
160:	learn: 0.7761122	total: 2m 34s	remaining: 37.5s
199:	learn: 0.7861653	total: 3m 11s	remaining: 0us
Метрика F1 на тестовой выборке: 0.7564


## Выводы

Используя подбор порога бинарной классификации задачу с приемлемой точностью (и относительно быстро) удается решить с помощью алгоритма логистической регрессии.